The Integration of Lasso and HHT in TS Forecasting

After HHT, we get
* A set of Hilbert transformed signals from IMFs
* Instantaneous frequency
* Instantaneous amplitude

which introduces sparsity, which means not all components carry useful predictive information but rather just noise-dominated and redundant

Lasso (L1 penalty) enforces sparsity by shrinking many coefficients exactly to 0 => perfect for HHT, as don't want all IMFs, just relevant ones. so Lasso will filters out noise features 

### Holt-Winter Triple Exponential smoothing forecasting

https://www.solarwinds.com/blog/holt-winters-forecasting-simplified

#### Pre-requisites: Exponential Smoothing (EMA)

When forecasting, add an error bands to show confidence level for future forecast

### Gray Forecasting

### Fuzzy Logic